# Connecting google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# A few necessary pip installs 

In [3]:
!pip install -qq tf2onnx
!pip install -qq onnxruntime-gpu 

     |████████████████████████████████| 440 kB 5.0 MB/s 
     |████████████████████████████████| 13.1 MB 40.0 MB/s 
     |████████████████████████████████| 111.0 MB 29 kB/s 
     |████████████████████████████████| 46 kB 2.6 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 


# Basic Imports for the transformation

In [4]:
from pathlib import Path 
import numpy as np

# Converting the keras model to onnx

First we load the keras model with its trained weights via the weights.h5 file.

Please visit:
https://drive.google.com/drive/u/1/folders/1fSjE_pz3jLTfmv7HsY7kFtivC-1a-c1b

If the notebook is in google colab with drive access.

In [5]:
from keras import Sequential
from keras.layers import Convolution2D
import tensorflow as tf


def get_model(weights_path=None):
    model = Sequential()
    model.add(
        Convolution2D(
            32, 9, activation="relu", input_shape=(400, 400, 3), padding="same"
        )
    )
    model.add(Convolution2D(16, 5, activation="relu", padding="same"))
    model.add(Convolution2D(3, 5, activation="relu", padding="same"))
    if weights_path and weights_path.is_file() and weights_path.suffix == '.h5':
        model.load_weights(weights_path)
    model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

    return model

############################################################################################### 
### Please make sure so that model_load_path is the path to the saved model weight.h5 file. ###
############################################################################################### 
drive_path = Path('/content/drive/.shortcut-targets-by-id/1fSjE_pz3jLTfmv7HsY7kFtivC-1a-c1b/BONDLY')
model_load_path = drive_path/'checkpoints/weights.h5'
print(model_load_path)

model = get_model(model_load_path)

/content/drive/.shortcut-targets-by-id/1fSjE_pz3jLTfmv7HsY7kFtivC-1a-c1b/BONDLY/checkpoints/weights.h5


# Conveting the keras model to onnx

Here we define the output file path in the `output_path` variable. The keraas model along with its weights are converted to onnx format via the `tf2onnx.convert.from_keras()` function. 

For example here the model is saved in the current working direcory under the name `srcnn.onnx`. The output path must end with the name of the onnx file with .onnx extention.

In [6]:
import tf2onnx
import onnxruntime as rt

############################################################ 
### Please make sure so that output_path ends with .onnx ###
############################################################ 
output_path = "srcnn.onnx"
spec = (tf.TensorSpec((None, 400, 400, 3), tf.float32, name="input"),)

model_proto, temp_ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]
output_names

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


['conv2d_2']

Here the `spec` variable is a demo input to the model. The `output_names` variable is a list of the output layers. Currently it is only holdes a single layer `['conv2d_2']`.

Next we create a demo onnx runtime Inference Session using `rt.InferenceSession()` function.

In [7]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession('srcnn.onnx', providers=providers)

Now we just create a demo numpy array with with the input shape of `(1,400,400,3)` and pass it as input to the Inference Session.  Finally the prediction is a list of output but as we gave a single input we have only one output in te list.

In [8]:
x = np.zeros((1,400,400,3),dtype="float32")
onnx_pred = m.run(['conv2d_2'], {"input": x})
onnx_pred[0]

array([[[[0.08867139, 0.09880005, 0.08365425],
         [0.08678415, 0.10349593, 0.08164187],
         [0.06368101, 0.09464593, 0.06262984],
         ...,
         [0.07277063, 0.10582839, 0.07981189],
         [0.08397173, 0.09884076, 0.08162892],
         [0.091378  , 0.09764111, 0.09485132]],

        [[0.10777766, 0.12211189, 0.11227132],
         [0.08606768, 0.1101473 , 0.0821394 ],
         [0.079318  , 0.1063973 , 0.06018153],
         ...,
         [0.10181407, 0.13181195, 0.08588666],
         [0.11194667, 0.12223721, 0.08834347],
         [0.10105506, 0.09582083, 0.08570465]],

        [[0.10799809, 0.11216333, 0.11326401],
         [0.07889979, 0.08541813, 0.06888802],
         [0.08662508, 0.09489053, 0.07217038],
         ...,
         [0.09495571, 0.11652584, 0.09766778],
         [0.10822386, 0.1163119 , 0.10548149],
         [0.10010433, 0.09536365, 0.09867963]],

        ...,

        [[0.09264387, 0.09401527, 0.10777704],
         [0.07939988, 0.07822873, 0.07552067]

# Part2: Using the onnx model to super-resolute any image
Now that we have the onnx model we will use that as an input to super-resolute any image of any size.

First we import a few basic libraries.

In [9]:
from tempfile import mkdtemp
import shutil
from pathlib import Path 
import gc
import numpy as np
import onnxruntime as rt
from PIL import Image
from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline

Now we declare a few helper functions and the main `super_resolution_whole_image()` function. This function takes in a pillow image, the onnx runtime Inference model and the up scale factor as input and gives out the super_resoluted image as output. 

On an average the function takes 1 min to finish each image.

In [26]:
def next_multiple(n,base):
    return int(base * np.ceil(n/base))

def memap_array_init(file_path,shape,dtype):
    np.save(file_path,np.array([]))
    temp_array = np.memmap(file_path, shape=shape, dtype=dtype)
    return temp_array

def plot_image(img):
    fig, ax = plt.subplots(figsize=(25,25))
    ax.imshow(np.asarray(img))
    plt.tight_layout()
    

def super_resolution_whole_image(img,model, up_res_scale = 2, padding = 10,frame = 400, tqdm_disable = True):
    rows = img.height
    cols = img.width

    prox_img_hight = next_multiple(rows*up_res_scale, frame-2*padding) + 2*padding
    prox_img_width = next_multiple(cols*up_res_scale, frame-2*padding) + 2*padding

    temp_dir_path = Path(mkdtemp())
    proxy_img = memap_array_init(file_path = temp_dir_path/'prox_img.npy',shape = (prox_img_hight,prox_img_width,3), dtype ='float32')
    out_image = memap_array_init(file_path = temp_dir_path/'out_img.npy',shape = (prox_img_hight,prox_img_width,3), dtype ='float32')
    
    proxy_img[padding:(rows*up_res_scale)+padding, padding:(cols*up_res_scale)+padding,:] = np.asarray(img.resize((cols*up_res_scale, rows*up_res_scale), Image.BILINEAR), dtype="float32")[:,:,:3]/ 255.0

    in_array = np.empty((1,frame,frame,3),dtype="float32")
    out_array = np.empty(in_array.shape,dtype="float32")

    tot = len(range(0,proxy_img.shape[0]-(frame-2*padding)+1,frame-2*padding))*len(range(0,proxy_img.shape[1]-(frame-2*padding)+1,frame-2*padding))
    with tqdm(range(tot),disable=tqdm_disable) as pbar:

        for i in range(0,proxy_img.shape[0]-(frame-2*padding)+1,frame-2*padding):
            for j in range(0,proxy_img.shape[1]-(frame-2*padding)+1,frame-2*padding):
                in_array[0] = proxy_img[i:i+frame,j:j+frame]
                out_array = model.run(['conv2d_2'], {"input": in_array})[0].clip(0,1)
                out_image[i+padding:i+frame-padding,j+padding:j+frame-padding] = np.uint8(out_array[0,padding:frame-padding,padding:frame-padding,:] * 255)
                pbar.update(1)
    
    out_image = out_image[padding:(rows*up_res_scale)+padding,padding:(cols*up_res_scale)+padding,:]

    if img.mode == 'RGBA':
        #print(out_image.shape)
        out_image = np.pad(out_image, ((0,0),(0,0),(0,1)), mode='constant', constant_values=0)
        out_image[:,:,-1] = np.asarray(img.resize((cols*up_res_scale, rows*up_res_scale), Image.BILINEAR), dtype="float32")[:,:,-1]
        #print(out_image.shape)
        #print(np.asarray(img.resize((cols*up_res_scale, rows*up_res_scale), Image.BILINEAR), dtype="float32").shape)

    
    out_image = Image.fromarray(np.uint8(out_image))

    del proxy_img
    gc.collect()
    shutil.rmtree(temp_dir_path)

    return out_image



Here we declare the .onnx file path in `model_load_path` and pass it to make an onnx session. Next we also load the image in a similar way and pass them both to the function and get the output image in the variable `super_res_img`.

In [11]:
drive_path = Path('/content/drive/.shortcut-targets-by-id/1fSjE_pz3jLTfmv7HsY7kFtivC-1a-c1b/BONDLY')

# This Path variable must lead to the onnx file to be used
model_load_path = Path("srcnn.onnx")
model = rt.InferenceSession(str(model_load_path), providers=['CPUExecutionProvider'])

# This Path must lead to the image to be loaded by the pillow Image class
image_path = drive_path/'Data/notree.jpg'
img = Image.open(image_path)

### pass tqdm_disable = False to show a progressbar
super_res_img = super_resolution_whole_image(img,model,padding = 10, tqdm_disable = False) 

100%|██████████| 352/352 [01:06<00:00,  5.30it/s]


In [12]:
#drive_path = Path('/content/drive/.shortcut-targets-by-id/13-O4CJbETG1lHoQin4ty3EwSVZ2-JHhC/georeferenced andramasina drone image')
drive_path = Path('/content/drive/.shortcut-targets-by-id/1NReAuS8nqleKi1EaxJDSyLbjt-_k5-sn/GeoData_Source')
tif_files = [tif for tif in drive_path.glob('**/*.tif')]
tif_files

[PosixPath('/content/drive/.shortcut-targets-by-id/1NReAuS8nqleKi1EaxJDSyLbjt-_k5-sn/GeoData_Source/Drone_Image_Points/Processed/Andramasina-Plot-1-20220616-orthophoto-RGB.tif'),
 PosixPath('/content/drive/.shortcut-targets-by-id/1NReAuS8nqleKi1EaxJDSyLbjt-_k5-sn/GeoData_Source/Drone_Image_Points/Processed/Andramasina-Plot-1-20220616-orthophoto-Raw.tif'),
 PosixPath('/content/drive/.shortcut-targets-by-id/1NReAuS8nqleKi1EaxJDSyLbjt-_k5-sn/GeoData_Source/Meteorological_Data/Wapor_Precipitation_Monthly/L1_PCP_2204M_clipped.tif'),
 PosixPath('/content/drive/.shortcut-targets-by-id/1NReAuS8nqleKi1EaxJDSyLbjt-_k5-sn/GeoData_Source/Meteorological_Data/Wapor_Precipitation_Monthly/L1_PCP_2203M_clipped.tif'),
 PosixPath('/content/drive/.shortcut-targets-by-id/1NReAuS8nqleKi1EaxJDSyLbjt-_k5-sn/GeoData_Source/Meteorological_Data/Wapor_Precipitation_Monthly/L1_PCP_2202M_clipped.tif'),
 PosixPath('/content/drive/.shortcut-targets-by-id/1NReAuS8nqleKi1EaxJDSyLbjt-_k5-sn/GeoData_Source/Meteorological

In [13]:
from PIL import UnidentifiedImageError
#import PIL
#PIL.Image.MAX_IMAGE_PIXELS = 605151464

rgba_tif_files = []

for i,file_ in enumerate(tif_files):
    try:
        img = Image.open(file_)
        print(i)
        if img.mode == 'RGBA' or img.mode == 'RGB':
            print(img.size,file_.name,file_.parts[-3]) 
            rgba_tif_files.append(file_)
    except UnidentifiedImageError:
        print(i, 'fail')
        pass
    #except DecompressionBombError:
    #    print("pass")
    

0
(1211, 1128) Andramasina-Plot-1-20220616-orthophoto-RGB.tif Drone_Image_Points
1
(1166, 1142) Andramasina-Plot-1-20220616-orthophoto-Raw.tif Drone_Image_Points
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
(520, 450) Andamasina_plot_9_orthophoto_14_06_2022.tif GeoData_Source
88
(836, 978) Andamasina_plot_7_orthophoto_14_06_2022.tif GeoData_Source
89
(606, 1102) Andamasina_plot_6_orthophoto_14_06_2022.tif GeoData_Source
90
(1021, 936) Andamasina_plot_5_orthophoto_14_06_2022.tif GeoData_Source
91
(865, 935) Andamasina_plot_4_orthophoto_14_06_2022.tif GeoData_Source
92
(683, 869) Andamasina_plot_3_orthophoto_14_06_2022.tif GeoData_Source
93
(2402, 2569) Andamasina_plot_4_orthophoto_high_resolution_14_06_2022.tif GeoData_Source
94
(2755, 2380) Andamasina_plot_5_orthophoto_high_resolution

DecompressionBombError: ignored

In [14]:
len(rgba_tif_files[:-3])
len(tif_files)

134

In [15]:
len(rgba_tif_files)
import PIL
#from PIL import Image
PIL.Image.MAX_IMAGE_PIXELS

89478485

In [28]:
#import PIL
#from PIL import Image
#PIL.Image.MAX_IMAGE_PIXELS = 605151464  #178956970
def plot_image(img):
    fig, ax = plt.subplots(figsize=(25,25))
    ax.imshow(np.asarray(img))
    plt.tight_layout()

def name(parts):
    name = parts[-1]
    if parts[1] == 'odm_orthophoto':
        name = parts[0]+'_'+name
    
    return name

output_path = drive_path/'Super_Resolution_Output'
print(output_path)
for i,t in enumerate(rgba_tif_files[:-3]):
    output_file_name = output_path/name(t.parts[-3:])
    img = Image.open(t)

    super_res_img = super_resolution_whole_image(img,model,padding = 10,up_res_scale=1,tqdm_disable=False)
    #output_path.mkdir(exist_ok = True)

    super_res_img.save(output_file_name)

/content/drive/.shortcut-targets-by-id/1NReAuS8nqleKi1EaxJDSyLbjt-_k5-sn/GeoData_Source/Super_Resolution_Output


100%|██████████| 12/12 [00:02<00:00,  4.12it/s]


(1128, 1211, 3)


100%|██████████| 16/16 [00:02<00:00,  6.05it/s]


(1142, 1166, 3)


...

(869, 683, 3)


100%|██████████| 9/9 [00:01<00:00,  6.09it/s]


(936, 1021, 3)


100%|██████████| 9/9 [00:01<00:00,  6.14it/s]


(935, 865, 3)


Next we use the helper functions to view and compare the input and output images.

In [ ]:
# Input image
print(img.size)
plot_image(img)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Output Image
print(super_res_img.size)
plot_image(super_res_img)

Output hidden; open in https://colab.research.google.com to view.

# Thank you